#Beam Element - This notebook computes the mass matrix and internal forces and outputs them for later use (v4 changed dimensions for 3D problem)

In [1]:
import numpy as np
import scipy as sp
import sympy as sym
import pickle

from scipy import linalg
from sympy import mpmath
from sympy import cos, sin

from IPython.display import display
from __future__ import division
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

#### Define Needed Functions

In [2]:
def Skew_sym(v):
    """
    This function returns the skew symetric matrix 
    of the vector 'v' to affect the cross product of 'v'x'u'
    """
    v_matrix = sym.Matrix([[  0 , -v[2],  v[1]],
                          [v[2],     0, -v[0]],
                          [-v[1],  v[0],    0]])
    return v_matrix

In [3]:
def Axial_sym(A):
    a_vec = 1/2*sym.Matrix([A[2,1] - A[1,2], A[0,2] - A[2,0], A[1,0] - A[0,1]])
    return a_vec

In [4]:
def Rotate_sym(theta):
    """
    This function returns the symbolic rotation matrix 
    for the simple 2D-rotation about the third axis
    """
    R = sym.Matrix([[cos(theta),-sin(theta), 0],
                    [sin(theta), cos(theta), 0],[0,0,1]])
    return R

#### Define symbolic quantites

In [5]:
# nodal positons, velocities, and accelerations
"""
"""

# symbolic system parameters 
E, I, A, rho, x, l, tau_a = sym.symbols('E I A rho x l, tau_a')

# Beam coordinates at cross-section 1 and 2
# r1 = sym.Matrix(['r_1x','r_1y'])
# r2 = sym.Matrix(['r_2x','r_2y'])
r1 = sym.Matrix(['0','0'])
r2 = sym.Matrix(['0','0'])


# Deformation coordinates (3D - x,y,z for ease of calculations later)
u_sym = sym.Matrix(['u_1x','u_1y','u_1z','u_2x','u_2y','u_2z'])
udot_sym = sym.Matrix(['udot_1x','udot_1y','udot_1z','udot_2x','udot_2y','udot_2z'])
uddot_sym = sym.Matrix(['uddot_1x','uddot_1y','uddot_1z','uddot_2x','uddot_2y','uddot_2z'])

# 1D Rotation at each note
theta = sym.Matrix(['theta_1','theta_2'])
thetadot = sym.Matrix(['thetadot_1','thetadot_2'])
thetaddot = sym.Matrix(['thetaddot_1','thetaddot_2'])

# locating the point in the cross section
s1 = sym.Matrix([0, r1])
s2 = sym.Matrix([0, r2])
s = sym.Matrix.vstack(s1,s2)

# Actual deformations z-component zero (kinematics) 
u = sym.Matrix([u_sym[0:2,0], '0', u_sym[3:5,0], '0'])
udot = sym.Matrix([udot_sym[0:2,0], '0', udot_sym[3:5,0], '0'])
uddot = sym.Matrix([uddot_sym[0:2,0], '0', uddot_sym[3:5,0], '0'])

# define generalized coordinates
q = sym.Matrix([u[0:2,0], [theta[0]], u[3:5,0], [theta[1]]])
qdot = sym.Matrix([udot[0:2,0], [thetadot[0]], udot[3:5,0], [thetadot[1]]])
qddot = sym.Matrix([uddot[0:2,0], [thetaddot[0]], uddot[3:5,0], [thetaddot[1]]])

display([q,qdot,qddot])
display([u,udot,uddot])

⎡⎡u₁ₓ ⎤, ⎡u̇₁ₓ ⎤, ⎡ü₁ₓ ⎤⎤
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u_1y⎥  ⎢u̇_1y⎥  ⎢ü_1y⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢ θ₁ ⎥  ⎢ θ̇₁ ⎥  ⎢ θ̈₁ ⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u₂ₓ ⎥  ⎢u̇₂ₓ ⎥  ⎢ü₂ₓ ⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u_2y⎥  ⎢u̇_2y⎥  ⎢ü_2y⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎣⎣ θ₂ ⎦  ⎣ θ̇₂ ⎦  ⎣ θ̈₂ ⎦⎦

⎡⎡u₁ₓ ⎤, ⎡u̇₁ₓ ⎤, ⎡ü₁ₓ ⎤⎤
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u_1y⎥  ⎢u̇_1y⎥  ⎢ü_1y⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢ 0  ⎥  ⎢  0  ⎥  ⎢  0  ⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u₂ₓ ⎥  ⎢u̇₂ₓ ⎥  ⎢ü₂ₓ ⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u_2y⎥  ⎢u̇_2y⎥  ⎢ü_2y⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎣⎣ 0  ⎦  ⎣  0  ⎦  ⎣  0  ⎦⎦

### Needed Matrix Quantities

In [6]:
# angular velocity for 2D case is trivial
omega1_skew = Skew_sym([0,0,thetadot[0]])
omega2_skew = Skew_sym([0,0,thetadot[1]])
alpha1_skew = Skew_sym([0,0,thetaddot[0]])
alpha2_skew = Skew_sym([0,0,thetaddot[1]])

# 1D Roation at each node
R1 = Rotate_sym(theta[0])
R2 = Rotate_sym(theta[1])
# display(R1,R2)

# "spatial" rotation matrix
R = sym.Matrix.vstack(sym.Matrix.hstack(R1,sym.zeros(3)), \
                      sym.Matrix.hstack(sym.zeros(3),R2))

# "spatial" angular velocity matrix
Omega_skew = sym.Matrix.vstack(sym.Matrix.hstack(omega1_skew,sym.zeros(3)), \
                               sym.Matrix.hstack(sym.zeros(3),omega2_skew))

# "spatial" angular acceleration matrix
Alpha_skew = sym.Matrix.vstack(sym.Matrix.hstack(alpha1_skew,sym.zeros(3)), \
                               sym.Matrix.hstack(sym.zeros(3),alpha2_skew))
# display(Omega)

### Define Kinematics

In [7]:
# Define velocity of element endpoints (nodes)
v = sym.simplify(udot + R*Omega_skew*s)
print('v = ')
display(v)

# Define acceleration of element endpoints (nodes)
a = sym.simplify(uddot + R*Omega_skew*Omega_skew*s + R*Alpha_skew*s)
print('\na = ')
display(a)

v = 


⎡u̇₁ₓ ⎤
⎢     ⎥
⎢u̇_1y⎥
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎢u̇₂ₓ ⎥
⎢     ⎥
⎢u̇_2y⎥
⎢     ⎥
⎣  0  ⎦


a = 


⎡ü₁ₓ ⎤
⎢     ⎥
⎢ü_1y⎥
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎢ü₂ₓ ⎥
⎢     ⎥
⎢ü_2y⎥
⎢     ⎥
⎣  0  ⎦

### Compute the Mass Matrix

In [8]:
# Define shape function for element with one node at each end
h = sym.symarray('h', 2)

h[0] = sym.Rational(1,2)*(1 - x)
h[1] = sym.Rational(1,2)*(1 + x)

# Compute shape function matrix
H = sym.expand(sym.Matrix([h[0]*sym.eye(3), h[1]*sym.eye(3)])).T
print('\nH = ')
display(H)

# Define velocity of any point 
Vp = H*v
print('\nV = ')
display(Vp)

# Define velocity of any point 
Ap = H*a
# print('\nA = ')
# display(Accel)

# Compute partial velocities of the nodes
Vr = sym.Matrix([[sym.diff(Vp,qdot) for Vp in Vp] for qdot in qdot]).T
# v_r = H
print('\nVr = ')
display(Vr)
# print(Vr.shape)

# Compute mass matrix
M = sym.factor(sym.Matrix([[sym.expand(sym.integrate(Vr[:,i].dot(Ap)*rho*A,(x,0,l))).coeff(qddot[j]) \
                            for i in range(len(qddot))] for j in range(len(qddot))]))
# print('\nM = \n')
# display(M)
pickle.dump( M, open( "mass-matrix.dump", "wb" ) )


H = 


⎡  x   1                    x   1              ⎤
⎢- ─ + ─     0        0     ─ + ─    0      0  ⎥
⎢  2   2                    2   2              ⎥
⎢                                              ⎥
⎢           x   1                  x   1       ⎥
⎢   0     - ─ + ─     0       0    ─ + ─    0  ⎥
⎢           2   2                  2   2       ⎥
⎢                                              ⎥
⎢                    x   1                x   1⎥
⎢   0        0     - ─ + ─    0      0    ─ + ─⎥
⎣                    2   2                2   2⎦


V = 


⎡      ⎛  x   1⎞        ⎛x   1⎞ ⎤
⎢ u̇₁ₓ⋅⎜- ─ + ─⎟ + u̇₂ₓ⋅⎜─ + ─⎟ ⎥
⎢      ⎝  2   2⎠        ⎝2   2⎠ ⎥
⎢                               ⎥
⎢      ⎛  x   1⎞         ⎛x   1⎞⎥
⎢u̇_1y⋅⎜- ─ + ─⎟ + u̇_2y⋅⎜─ + ─⎟⎥
⎢      ⎝  2   2⎠         ⎝2   2⎠⎥
⎢                               ⎥
⎣               0               ⎦


Vr = 


⎡  x   1              x   1          ⎤
⎢- ─ + ─     0     0  ─ + ─    0    0⎥
⎢  2   2              2   2          ⎥
⎢                                    ⎥
⎢           x   1            x   1   ⎥
⎢   0     - ─ + ─  0    0    ─ + ─  0⎥
⎢           2   2            2   2   ⎥
⎢                                    ⎥
⎣   0        0     0    0      0    0⎦

In [9]:
print('M_11 = ')
display(M[0:4,0:4])

print('\nM_22 = ')
display(M[4:8,4:8])

print('\nM_12 = ')
display(M[0:4,4:8])
print('\nM_21.T = ')
display(M[4:8,0:4].T)

M_11 = 


⎡      ⎛ 2          ⎞                                    ⎛ 2    ⎞   ⎤
⎢A⋅l⋅ρ⋅⎝l  - 3⋅l + 3⎠                             -A⋅l⋅ρ⋅⎝l  - 3⎠   ⎥
⎢────────────────────           0            0    ────────────────  ⎥
⎢         12                                             12         ⎥
⎢                                                                   ⎥
⎢                            ⎛ 2          ⎞                         ⎥
⎢                      A⋅l⋅ρ⋅⎝l  - 3⋅l + 3⎠                         ⎥
⎢         0            ────────────────────  0           0          ⎥
⎢                               12                                  ⎥
⎢                                                                   ⎥
⎢         0                     0            0           0          ⎥
⎢                                                                   ⎥
⎢         ⎛ 2    ⎞                                    ⎛ 2          ⎞⎥
⎢  -A⋅l⋅ρ⋅⎝l  - 3⎠                              A⋅l⋅ρ⋅⎝l  + 3⋅l + 3⎠⎥
⎢  ──────────────── 


M_22 = 


⎡      ⎛ 2          ⎞   ⎤
⎢A⋅l⋅ρ⋅⎝l  + 3⋅l + 3⎠   ⎥
⎢────────────────────  0⎥
⎢         12            ⎥
⎢                       ⎥
⎣         0            0⎦


M_12 = 


⎡       0          0⎤
⎢                   ⎥
⎢       ⎛ 2    ⎞    ⎥
⎢-A⋅l⋅ρ⋅⎝l  - 3⎠    ⎥
⎢────────────────  0⎥
⎢       12          ⎥
⎢                   ⎥
⎢       0          0⎥
⎢                   ⎥
⎣       0          0⎦


M_21.T = 


⎡       0          0⎤
⎢                   ⎥
⎢       ⎛ 2    ⎞    ⎥
⎢-A⋅l⋅ρ⋅⎝l  - 3⎠    ⎥
⎢────────────────  0⎥
⎢       12          ⎥
⎢                   ⎥
⎢       0          0⎥
⎢                   ⎥
⎣       0          0⎦

### Compute Internal forces 

#### Transverse (Bending) Strain

In [10]:
''' Interpolate Finite Rotation Between Nodes (Interpolation of rotation and motion - Olivier A. Bauchau · Shilei Han)
    - Algorithm 2 (Cayley’s decomposition) is used to Extract Orthogonal Rotation Matrix R
    - Algorithm 4 is used to interpolate curvature '''
T = sym.simplify(H*sym.Matrix([R1,R2]))
# display(T)
A = (T + sym.eye(3))*(T + sym.eye(3)).inv()
# display(A)
cpar = Axial_sym(A)
# display(cpar)
temp = cpar.T*cpar
cpar_skew = Skew_sym(cpar)
R = 1/(temp[0]+1)*((1+temp[0])*sym.eye(3) + 2*cpar_skew + 2*cpar_skew*cpar_skew)
dT = h[0].diff(x)*R1 + h[1].diff(x)*R2
# display(dT)
kappa = sym.Matrix([Axial_sym(dT*R.T), '0', '0', '0'])
# display(kappa)

In [11]:
display(kappa)

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢    ⎛                                              ⎛      ⎛(-x + 1)⋅sin(θ₁)  
⎢    ⎜                                              ⎜    2⋅⎜──────────────── +
⎢    ⎜         ⎛(-x + 1)⋅sin(θ₁)   (x + 1)⋅sin(θ₂)⎞ ⎜      ⎝       2          
⎢0.5⋅⎜- - -1.0⋅⎜──────────────── + ───────────────⎟⋅⎜─────────────────────────
⎢    ⎜         ⎝       2                  2       ⎠ ⎜                         
⎢    ⎜                                              ⎜⎛                        
⎢    ⎜                                              ⎜⎝((x - 1)⋅sin(θ₁) - (x + 
⎢    ⎝                                              

#### Longitudinal (Axial) Strian

In [12]:
x01 = sym.Matrix(['0','0','0'])
x02 = sym.Matrix(['1','0','0'])
i_bar = sym.Matrix(['1','0','0'])

u_int = H*u
x_int = H*sym.Matrix([x01,x02])
# display(x_int)

du = u_int.diff(x)
dx0 = x_int.diff(x)

epsilon = sym.Matrix([dx0 + du - R*i_bar, '0', '0', '0'])
# display(epsilon)

#### Compute Internal Forces $Q_e = \frac{\partial U}{\partial e}$

In [ ]:
# Transverse strain energy
Ut = 1/2*sym.integrate(E*I*kappa.dot(H.diff(x).T*H.diff(x)*kappa), (x,0,l))

In [ ]:
# Longitudinal strain energy
Ul = 1/2*sym.integrate(E*A*epsilon.dot(H.diff(x).T*H.diff(x)*epsilon), (x,0,l))

In [ ]:
# Compute Total Energy
U = Ul + Ut

# Compute Internal Force Vector
Qe = sym.Matrix([sym.simplify(sym.expand(sym.diff(-U,q))) for q in q])

### Applied and body force vector

In [ ]:
# Applied forces
# Gravity body force
fg = rho*9.81*A*sym.Matrix([0,-1,0])

# Applied torque
# torque_app = sym.Matrix([0,0,tau_a])

# Compute beta
# beta = sym.Matrix([sym.simplify(sym.integrate(vr.dot(fg), (x,0,l)) + qe) 
#                    for vr,qe in zip(V_r,Qe)])
beta = sym.Matrix([sym.simplify(sym.integrate(V_r[:,j].dot(fg), (x,0,l)))
                   + qe for j,qe in zip(range(len(e)),Qe)])
pickle.dump( beta, open( "force-vector.dump", "wb" ) )